In [1]:
import cv2
import math
import numpy as np
import dlib

# 模型初始化
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 读取图像
#srcImg=cv2.imread("Images/tank.jpg")
image = cv2.imread("Images/tank.jpg")

# 色彩空间转换彩色(BGR)-->灰度（Gray）
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 获取人脸
faces = detector(gray, 3)
# 对检测到的rects，逐个遍历

for face in faces:
    # 针对脸部的关键点进行处理，构成坐标(x,y)形式
    shape = np.matrix([[p.x, p.y] for p in predictor(gray, face).parts()])
    shape=np.array(shape)
    
import matplotlib.pyplot as plt
def CalculateDistance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1]))

def localTranslationWarpFastWithStrength(srcImg, startX, startY, endX, endY, radius, strength):
    ddradius = float(radius * radius)
    copyImg = np.zeros(srcImg.shape, np.uint8)
    copyImg = srcImg.copy()
 
 
    maskImg = np.zeros(srcImg.shape[:2], np.uint8)
    cv2.circle(maskImg, (startX, startY), math.ceil(radius), (255, 255, 255), -1)
 
    K0 = 100/strength
 
    # 计算公式中的|m-c|^2
    ddmc_x = (endX - startX) * (endX - startX)
    ddmc_y = (endY - startY) * (endY - startY)
    H, W, C = srcImg.shape
 
    x1 = np.vstack([np.arange(W).astype(np.float32).reshape(1, -1)] * H)
    y1 = np.hstack([np.arange(H).astype(np.float32).reshape(-1, 1)] * W)

    # 计算k1
    distance_x = (x1 - startX) * (x1 - startX)
    distance_y = (y1 - startY) * (y1 - startY)
    distance = distance_x + distance_y
    K1 = np.sqrt(distance)


    ratio_x = (ddradius - distance_x) / (ddradius - distance_x + K0 * ddmc_x)
    ratio_y = (ddradius - distance_y) / (ddradius - distance_y + K0 * ddmc_y)

    x_t = ratio_x**2* (endX - startX)
    y_t = ratio_y**2 * (endY - startY) 


    x2 = x1 - x_t * (1 - K1/radius)
    y2 = y1 - y_t* (1 - K1/radius)
 
    np.copyto(x2, x1, where=maskImg == 0)
    np.copyto(y2,y1, where=maskImg == 0)
    x2 = x2.astype(np.float32)
    y2 = y2.astype(np.float32)
    copyImg = cv2.remap(srcImg, x2, y2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
 
    return copyImg

 
processed_image = image.copy()
startX_left, startY_left, endX_left, endY_left = shape[2][0], shape[2][1], shape[30][0], shape[30][1]
startX_right, startY_right, endX_right, endY_right = shape[14][0],shape[14][1], shape[30][0], shape[30][1]
startU_left, startV_left, endU_left, endV_left = shape[6][0], shape[6][1], shape[30][0], shape[30][1]
startU_right, startV_right, endU_right, endV_right = shape[10][0],shape[10][1], shape[30][0], shape[30][1]

#radius1 = max(CalculateDistance(shape[30], shape[2]),CalculateDistance(shape[30],shape[14]))
radius1= CalculateDistance(shape[30], shape[2])
radius2 = CalculateDistance(shape[30],shape[14])
radius3 = CalculateDistance(shape[57],shape[8])

print("radius="+str(radius1))
strength  = 40
strength1 = 40
strength2 = 40
strength3 = 50


Start_Left=  np.zeros((6,2),np.uint16)        # 6个关键点
for i in range(0,6):
    Start_Left[i] = shape[i+2]


Start_Right=  np.zeros((6,2),np.uint16)       # 6个关键点
j = 14
for i in range(0,6):                     # range 函数 range(Start_pos,End_pos,Step)
    Start_Right[i] = shape[j]
    j = j-1
print(Start_Right)

# 瘦下巴
processed_image = localTranslationWarpFastWithStrength(processed_image, shape[8][0], shape[8][1]+20, shape[57][0], shape[57][1],radius3, strength3)

# 瘦左边脸   
for i in range(0,6):
    processed_image = localTranslationWarpFastWithStrength(processed_image, Start_Left[i][0], Start_Left[i][1], endX_left, endY_left, CalculateDistance(Start_Left[i], shape[30]), strength/((i+1)**2))
# 瘦右边脸   
for i in range(0,6):
    processed_image = localTranslationWarpFastWithStrength(processed_image, Start_Right[i][0], Start_Right[i][1], endX_right, endY_right, CalculateDistance(Start_Right[i],shape[30]), strength/((i+1)**2))





# processed_image = localTranslationWarpFastWithStrength(processed_image, startX_left, startY_left, endX_left, endY_left, radius1, strength1)
# processed_image = localTranslationWarpFastWithStrength(processed_image, startU_left, startV_left, endX_left, endY_left, radius2/2, strength2/2)


# processed_image = localTranslationWarpFastWithStrength(processed_image, startX_right, startY_right, endX_right, endY_right, radius1, strength1)
# processed_image = localTranslationWarpFastWithStrength(processed_image, startU_right, startV_right, endU_right, endV_right, radius2/2, strength2/2)


cv2.imshow("origin.jpg", image)
cv2.imwrite("origin.jpg", image)
cv2.imshow("thin.jpg", processed_image)
cv2.imwrite("thin.jpg", processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite("thin.jpg", processed_image)

radius=164.04877323527901
[[409 272]
 [402 314]
 [387 355]
 [364 390]
 [329 416]
 [291 435]]
